# primary model comparison

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import preprocessing
from sklearn.impute import SimpleImputer

# model evaluation
from sklearn.metrics import classification_report, f1_score, accuracy_score

# classifiers
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.tree import DecisionTreeClassifier # decision tree
from sklearn.ensemble import RandomForestClassifier # random forest
from sklearn.ensemble import GradientBoostingClassifier # gradient boosting

## data preprocess

### read train.csv, preprocess

In [2]:
# load data & show head
df = pd.read_csv('../cs5228/train.csv')
print(df.shape)
df.head()

(24421, 14)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,exceeds50K
0,30,?,147215,Some-college,10,Never-married,?,Own-child,Female,0,0,30,United-States,0
1,60,Private,173960,Bachelors,13,Divorced,Prof-specialty,Not-in-family,Female,0,0,42,United-States,0
2,52,?,105428,Some-college,10,Married-civ-spouse,?,Husband,Male,0,0,12,United-States,0
3,37,Private,112497,Bachelors,13,Married-civ-spouse,Sales,Husband,Male,0,0,60,United-States,0
4,63,Private,137843,Some-college,10,Married-civ-spouse,Sales,Husband,Male,7298,0,48,United-States,1


In [3]:
# split features and labels
X = df.drop(['exceeds50K'],axis=1)
Y = df['exceeds50K']
X.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country
0,30,?,147215,Some-college,10,Never-married,?,Own-child,Female,0,0,30,United-States
1,60,Private,173960,Bachelors,13,Divorced,Prof-specialty,Not-in-family,Female,0,0,42,United-States
2,52,?,105428,Some-college,10,Married-civ-spouse,?,Husband,Male,0,0,12,United-States


### read test.csv, preprocess

In [4]:
df_test = pd.read_csv('../cs5228/test.csv')
print(df_test.shape)
X_test = df_test
df_test.head(3)

(24421, 13)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country
0,23,Private,32732,Some-college,10,Married-civ-spouse,Sales,Husband,Male,0,0,25,United-States
1,69,Private,165017,HS-grad,9,Widowed,Machine-op-inspct,Unmarried,Male,2538,0,40,United-States
2,27,Private,36440,Bachelors,13,Never-married,Sales,Not-in-family,Female,0,0,40,United-States


In [5]:
# replace missing value ' ?' by mode

feature_list = X.columns

# count missing value ' ?' 
print('missing value count in train.scv')
for feature in feature_list:
    print(feature, ' ', len(X[X[feature] == ' ?']))

print('\nmissing value count in test.csv')
for feature in feature_list:
    print(feature, ' ', len(X_test[X_test[feature]==' ?']))

# replace
X_all = pd.concat([X,X_test], axis = 0)
imputer = SimpleImputer(missing_values=' ?', strategy='most_frequent')
imputer.fit(X_all)
X = pd.DataFrame(imputer.transform(X), columns = feature_list)
X_test = pd.DataFrame(imputer.transform(X_test), columns = feature_list)
X.head(3)

missing value count in train.scv
age   0
workclass   1392
fnlwgt   0
education   0
education-num   0
marital-status   0
occupation   1399
relationship   0
sex   0
capital-gain   0
capital-loss   0
hours-per-week   0
native-country   410

missing value count in test.csv
age   0
workclass   1407
fnlwgt   0
education   0
education-num   0
marital-status   0
occupation   1410
relationship   0
sex   0
capital-gain   0
capital-loss   0
hours-per-week   0
native-country   447


/Users/zyw/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country
0,30,Private,147215,Some-college,10,Never-married,Prof-specialty,Own-child,Female,0,0,30,United-States
1,60,Private,173960,Bachelors,13,Divorced,Prof-specialty,Not-in-family,Female,0,0,42,United-States
2,52,Private,105428,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,Male,0,0,12,United-States


In [6]:
# marital-status into single, married
print(set(X['marital-status'].values))
single_list = [' Separated', ' Divorced', ' Widowed', ' Never-married']
married_list = [' Married-AF-spouse', ' Married-spouse-absent', ' Married-civ-spouse']

X['marital-status'][X['marital-status'].isin(married_list)] = 'married'
X['marital-status'][X['marital-status'].isin(single_list)] = 'single'
X_test['marital-status'][X_test['marital-status'].isin(married_list)] = 'married'
X_test['marital-status'][X_test['marital-status'].isin(single_list)] = 'single'
X.head()

{' Separated', ' Never-married', ' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Widowed'}


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country
0,30,Private,147215,Some-college,10,single,Prof-specialty,Own-child,Female,0,0,30,United-States
1,60,Private,173960,Bachelors,13,single,Prof-specialty,Not-in-family,Female,0,0,42,United-States
2,52,Private,105428,Some-college,10,married,Prof-specialty,Husband,Male,0,0,12,United-States
3,37,Private,112497,Bachelors,13,married,Sales,Husband,Male,0,0,60,United-States
4,63,Private,137843,Some-college,10,married,Sales,Husband,Male,7298,0,48,United-States


### embedding, train&test data together

In [7]:
# transfer string feature into int

# features in string to be encoded
encoding_features = ['workclass','education','marital-status','occupation','relationship','sex','native-country']

# to include all possible values
X_all = pd.concat([X,X_test], axis = 0)
for feature in encoding_features:
    encoder = preprocessing.LabelEncoder()
    encoder.fit(X_all[feature])
    X[feature] = encoder.transform(X[feature])
    X_test[feature] = encoder.transform(X_test[feature])

X.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country
0,30,3,147215,15,10,1,9,3,0,0,0,30,38
1,60,3,173960,9,13,1,9,1,0,0,0,42,38
2,52,3,105428,15,10,0,9,0,1,0,0,12,38


In [8]:
# feature scaling

scaler = preprocessing.StandardScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns = feature_list)
X_test = pd.DataFrame(scaler.transform(X_test), columns = feature_list)
X.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country
0,-0.627855,-0.086978,-0.397416,1.218346,-0.037490,0.946594,0.714447,0.965669,-1.421717,-0.145954,-0.216863,-0.849954,0.263597
1,1.565644,-0.086978,-0.141947,-0.335686,1.127814,0.946594,0.714447,-0.279387,-1.421717,-0.145954,-0.216863,0.120349,0.263597
2,0.980711,-0.086978,-0.796568,1.218346,-0.037490,-1.056419,0.714447,-0.901915,0.703375,-0.145954,-0.216863,-2.305408,0.263597


In [9]:
X_train ,X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3, random_state = 0)

## model comparison

In [10]:
# Models Options: K-nearest neighbors, Logistic regression, Decision trees
#                 Random forest, Gradient boosting machine
model_names = ['KNN', 'LR', 'DT', 'RF', 'GBM']
df_acc = []
df_f1 = []


knn = KNeighborsClassifier()
lr = LogisticRegression()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
gbm = GradientBoostingClassifier()

for classifier in (knn, lr, dt, rf, gbm):
        classifier.fit(X_train, Y_train)
        Y_pred = classifier.predict(X_val)
        
        f1 = round(f1_score(Y_val, Y_pred, average='weighted') * 100, 2)
        acc = round(accuracy_score(Y_val, Y_pred) * 100, 2)
        
        df_acc.append(f1)
        df_f1.append(acc)
        
accuracy_record = pd.DataFrame({'Model': model_names, 'df_acc': df_acc})
accuracy_record.set_index('Model', inplace=True)

F1_record = pd.DataFrame({'Model': model_names, 'df_f1': df_f1})
F1_record.set_index('Model', inplace=True)

print(accuracy_record)
print('\n')
print(F1_record)

       df_acc
Model        
KNN     82.41
LR      83.88
DT      81.34
RF      85.42
GBM     86.15


       df_f1
Model       
KNN    82.86
LR     84.58
DT     81.36
RF     85.85
GBM    86.80
